In [ ]:
import csv                               # csv reader
from sklearn.svm import LinearSVC
from nltk.classify import SklearnClassifier
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from IPython.display import display
from nltk.corpus import stopwords
from random import shuffle
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from collections import Counter
import numpy
import string

In [ ]:
# load data from a file and append it to the rawData
def loadData(path, Text=None):
    with open(path, encoding="utf8") as f:
        reader = csv.reader(f, delimiter='\t')
        next(reader, None)
        for line in reader:
            (Id, Text, Rating, Verified, ProductID, Label) = parseReview(line)
            rawData.append((Id, Text, Rating, Verified, ProductID, Label))
            preprocessedData.append((Id, preProcess(Text, Rating, Verified, ProductID), Label))

def splitData(percentage):
    dataSamples = len(rawData)
    halfOfData = int(len(rawData)/2)
    trainingSamples = int((percentage*dataSamples)/2)
    for (_, Text, Rating, Verified, ProductID, Label) in rawData[:trainingSamples] + rawData[halfOfData:halfOfData+trainingSamples]:
        trainData.append((toFeatureVector(preProcess(Text, Rating, Verified, ProductID)),Label))
    for (_, Text, Rating, Verified, ProductID, Label) in rawData[trainingSamples:halfOfData] + rawData[halfOfData+trainingSamples:]:
        testData.append((toFeatureVector(preProcess(Text, Rating, Verified, ProductID)),Label))

# Question 1

In [ ]:
# Convert line from input file into an id/text/label tuple
def parseReview(reviewLine):
  ID = 0
  TEXT = 8
  LABEL = 1
  RATING = 2
  VERIFIED_PURCHASE = 3
  PRODUCT_ID = 5
  tuple = (int(reviewLine[ID]), reviewLine[TEXT], int(reviewLine[RATING]), True if reviewLine[VERIFIED_PURCHASE] == 'Y' else False, reviewLine[PRODUCT_ID], fakeLabel if reviewLine[LABEL]=='__label1__' else realLabel)
  return tuple
    # Should return a triple of an integer, a string containing the review, and a string indicating the label
    # DESCRIBE YOUR METHOD IN WORDS
    # return (None, None, None)

In [ ]:
# TEXT PREPROCESSING AND FEATURE VECTORIZATION

# Input: a string of one review
def preProcess(text, rating, verified, product_id):
    # Should return a list of tokens
    tokens = text.split(" ")

    # Stemming and lemmatisation
    porter_stemmer = PorterStemmer()
    wordnet_lemmatizer = WordNetLemmatizer()


    # Removing the stop word. You will need to run nltk.download("stopwords") in python console.
    stop_words = set(stopwords.words('english'))
    tokens = [wordnet_lemmatizer.lemmatize(porter_stemmer.stem(w)) for w in tokens if w not in stop_words]
    tokens = [w.translate(str.maketrans('', '', string.punctuation)) for w in tokens if w]
    return (tokens, rating, verified, product_id)
    # Should return a list of tokens
    # DESCRIBE YOUR METHOD IN WORDS
    # return []

# Question 2

In [ ]:
featureDict = {} # A global dictionary of features

def toFeatureVector(tokens):
  featurevect = dict(Counter(tokens[0]))
  featurevect['rating'] = tokens[1]
  featurevect['verified'] = 1 if tokens[2] else 0
  featurevect['productID'+tokens[3]] = 1
  return featurevect
    # Should return a dictionary containing features as keys, and weights as values
    # DESCRIBE YOUR METHOD IN WORDS

In [ ]:
# TRAINING AND VALIDATING OUR CLASSIFIER
def trainClassifier(trainData):
    print("Training Classifier...")
    pipeline =  Pipeline([('svc', LinearSVC())])
    return SklearnClassifier(pipeline).train(trainData)

# Question 3

In [ ]:
def crossValidate(dataset, folds):
    shuffle(dataset)
    cv_results = []
    foldSize = int(len(dataset)/folds)
    # DESCRIBE YOUR METHOD IN WORDS
    for i in range(0,len(dataset),foldSize):
       testFold = dataset[i:i+foldSize] # This our fold used for testing.
       trainingData = dataset[0:i] + dataset[foldSize:]
       classifier = trainClassifier(trainingData)
       y_pred = predictLabels(testFold, classifier)
       y_true = list(map(lambda t: t[1], testFold))
       results = list(precision_recall_fscore_support(y_true, y_pred, average='weighted'))
       results[3] = accuracy_score(y_true, y_pred) * 100
       cv_results.append(tuple(results))

    return cv_results

In [ ]:
# PREDICTING LABELS GIVEN A CLASSIFIER

def predictLabels(reviewSamples, classifier):
    return classifier.classify_many(map(lambda t: t[0], reviewSamples))

def predictLabel(reviewSample, classifier):
    return classifier.classify(toFeatureVector(preProcess(reviewSample)))

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from sklearn import preprocessing
# MAIN

# loading reviews
# initialize global lists that will be appended to by the methods below
rawData = []          # the filtered data from the dataset file (should be 21000 samples)
preprocessedData = []
trainData = []        # the pre-processed training data as a percentage of the total dataset (currently 80%, or 16800 samples)
testData = []         # the pre-processed test data as a percentage of the total dataset (currently 20%, or 4200 samples)

# the output classes
fakeLabel = 'fake'
realLabel = 'real'

# references to the data files
reviewPath = 'amazon_reviews.txt'

# Do the actual stuff (i.e. call the functions we've made)
# We parse the dataset and put it in a raw data list
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing the dataset...",sep='\n')
loadData(reviewPath)

# We split the raw dataset into a set of training data and a set of test data (80/20)
# You do the cross validation on the 80% (training data)
# We print the number of training samples and the number of features before the split
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing training and test data...",sep='\n')
splitData(0.8)
# We print the number of training samples and the number of features after the split
print("After split, %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Training Samples: ", len(trainData), "Features: ", len(featureDict), sep='\n')
cvResults = crossValidate(trainData, 10)
display(cvResults)

print("Done!")
# QUESTION 3 - Make sure there is a function call here to the
# crossValidate function on the training set to get your results


Now 0 rawData, 0 trainData, 0 testData
Preparing the dataset...
Now 21000 rawData, 0 trainData, 0 testData
Preparing training and test data...
After split, 21000 rawData, 16800 trainData, 4200 testData
Training Samples: 
16800
Features: 
0
Training Classifier...


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Training Classifier...


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Training Classifier...


...

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Training Classifier...


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[(0.7780159293051595,
  0.7773809523809524,
  0.7773411962187473,
  77.73809523809524),
 (1.0, 1.0, 1.0, 100.0),
 (1.0, 1.0, 1.0, 100.0),
 (1.0, 1.0, 1.0, 100.0),
 (1.0, 1.0, 1.0, 100.0),
 (1.0, 1.0, 1.0, 100.0),
 (1.0, 1.0, 1.0, 100.0),
 (1.0, 1.0, 1.0, 100.0),
 (1.0, 1.0, 1.0, 100.0),
 (1.0, 1.0, 1.0, 100.0)]

Done!


# Evaluate on test set

In [ ]:
# Finally, check the accuracy of your classifier by training on all the traning data
# and testing on the test set
# Will only work once all functions are complete
functions_complete = True  # set to True once you're happy with your methods for cross val
if functions_complete:
    print(testData[0])   # have a look at the first test data instance
    classifier = trainClassifier(trainData)  # train the classifier
    testTrue = [t[1] for t in testData]   # get the ground-truth labels from the data
    testPred = predictLabels(testData, classifier)  # classify the test data to get predicted labels
    finalScores = precision_recall_fscore_support(testTrue, testPred, average='weighted') # evaluate
    print("Done training!")
    print("Precision: %f\nRecall: %f\nF Score:%f" % finalScores[:3])

({'thi': 1, 'assort': 1, 'realli': 1, 'hershey': 1, 'best': 1, 'the': 1, 'littl': 1, 'one': 1, 'alway': 1, 'excit': 1, 'whenev': 1, 'holiday': 1, 'come': 1, 'this': 1, 'rating': 5, 'verified': 0, 'productIDB004B9T82W': 1}, 'fake')
Training Classifier...


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Done training!
Precision: 0.766247
Recall: 0.765476
F Score:0.765306


# Questions 5
Key changes:

* Preprocessing of reviews has been altered to ensure proper tokenisation, stemming, and lemmatisation.
* The toFeatureVector function was altered to include rating, verified purchase status, and product ID.
* Added prefixes in the toFeatureVector function for differentiation between product IDs and other tokens. This will make the feature space more explicit and less prone to overlaps.